In [6]:
# Import Pandas
import pandas as pd

#Set Relative Path
import sys
sys.path.append("../src")

In [7]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/CleanKaggle.csv')
df_dataK.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from wordcloud import STOPWORDS
STOPWORDS.update(['feel','feeling','im',",","t","u","2","'","&amp;","-","...","s"])

# Set X and y
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=STOPWORDS)
tfidf_matrix = tf.fit_transform(df_dataK['text'])
X = tfidf_matrix

Labelenc = LabelEncoder()
df_dataK['emotion'] = Labelenc.fit_transform(df_dataK['emotion'])
y = df_dataK['emotion']
print (y.unique())

/home/apprenant/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amp', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
[4 0 3 5 1 2]


In [9]:
# Prediction
#userinput= str(input()) 
userinput = "i love you so much!!!!"
print (userinput)
x = tf.transform([userinput])

i love you so much!!!!


In [10]:
from reg_lin import get_metrix
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential 
from tensorflow.keras import layers, preprocessing
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score
models = [  LogisticRegression(),
            SGDClassifier(),
            RandomForestClassifier(),
            XGBClassifier(),
            Sequential()]
place = 4
savedmodel=[]
count = 0
for modelset in models :
    if count == place:
        
        df_dataKKeras = df_dataK[['text']].copy()
        tk = preprocessing.text.Tokenizer()
        tk.fit_on_texts(df_dataKKeras['text'])
        X = tk.texts_to_matrix(df_dataKKeras['text'], mode='tfidf')
        x = tk.texts_to_matrix([userinput], mode='tfidf')
      
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=None)
        model = modelset
        model.add(layers.Dense(160, activation = 'relu', input_shape= [X_train.shape[1]]))
        model.add(layers.Dense(80, activation = 'relu'))
        model.add(layers.Dense(40, activation='relu'))
        model.add(layers.Dense(1,activation='softmax'))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy']) 
        model.fit(X_train,y_train,epochs=10, batch_size=100)

        y_train_pred = model.predict_classes(X_train)
        y_test_pred = model.predict_classes(X_test)
        
        print (model)

        print (" Train accuracy = {}" 
        .format(round(accuracy_score(y_train, y_train_pred),3)))

        print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))
    if count != place:
        model = get_metrix(y,X,modelset)
    pred = model.predict(x)
    print (pred)
    if count == place:
        pred = Labelenc.inverse_transform([round(pred[0][0])])
    if count != place:
        pred = Labelenc.inverse_transform(pred)
    print (pred)
    savedmodel.append(model)
    count += 1
    print("\n")

/home/apprenant/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
LogisticRegression()
Train R2 = 0.881; Train RMSE = 0.504; Test accuracy = 0.953
Test R2  = 0.454; Test RMSE = 1.096; Test accuracy = 0.781
[2]
['happy']


SGDClassifier()
Train R2 = 0.967; Train RMSE = 0.267; Test accuracy = 0.989
Test R2  = 0.675; Test RMSE = 0.846; Test accuracy = 0.883
[2]
['happy']


RandomForestClassifier()
Train R2 = 0.992; Train RMSE = 0.129; Test accuracy = 0.997
Test R2  = 0.671; Test RMSE = 0.844; Test accuracy = 0.886
[2]
['happy']


/home/ap